In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv')

In [4]:
dataset.head(10)

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0

In [5]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [6]:
for name, group in dataset.groupby('user'):
    print(name,len(group)) 

U1 2470
U10 531
U11 167
U12 820
U13 90
U2 271
U3 331
U4 227
U5 17
U6 279
U7 397
U8 214
U9 79


In [7]:
dataset.target.unique().tolist() #all users

['Still', 'Car', 'Train', 'Bus', 'Walking']

In [8]:
dataset.target = dataset.target.replace({'Still':'Not Walking','Train':'Not Walking','Car':'Not Walking','Bus':'Not Walking'})

In [9]:
for name, group in dataset.groupby('target'):
    print(name,len(group)) 

Not Walking 4716
Walking 1177


In [10]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [11]:
#dataset.columns

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [12]:
#dataset.info()

In [13]:
dataset.describe()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.0000,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,2946.0000,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,1701.3069,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.

In [14]:
dataset.isnull().sum()

Unnamed: 0                                            0
id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                   

In [15]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-15-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,Unnamed: 0,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Not Walking,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Not Walking,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Not Walking,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Not Walking,U12
4,4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.7707

In [16]:
dataset['rv_gyro_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']
dataset['lin_speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']
dataset['rv_gyro_prox_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['proximity_std']
dataset['lin_speed_prox_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean'] *dataset['proximity_std']
dataset['rv_gyro__grv_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['game_rotation_vector_mean']

In [17]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'sound_max',
                    'sound_mean',
                    
                    'rv_gyro_mean',
                    
                    'lin_speed_mean',
                    'rv_gyro_prox_mean',
                    'lin_speed_prox_mean',
                    'rv_gyro__grv_mean',
                    
                    'user',
                    'target'
                 ])]

In [18]:
dataset.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean', 'user', 'target'],
      dtype='object')

In [19]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.000737,0.020978,0.050413,0.056351,NaN,51.199707,NaN,NaN,NaN,0.000083,0.000000,NaN,NaN,0.000002,U12,Not Walking
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.020290,0.879220,0.999981,0.999999,NaN,82.409890,89.065143,89.335277,89.200210,0.036325,14.541729,NaN,NaN,0.036322,U12,Not Walking
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.000713,0.034110,0.610456,0.610456,NaN,55.501802,NaN,NaN,NaN,0.000931,0.000000,NaN,NaN,0.000619,U12,Not Walking
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.010553,1.488361,0.998112,0.998112,NaN,95.664309,87.470377,87.470377,87.470377,0.036372,26.403360,NaN,NaN,0.036234,U12,Not Walking
4,8.965621,10.891645,0.504117,0.183202,0.265652,0.098819,1.658308,0.322242,0.378193,NaN,156.795909,89.770732,89.770732,89.770732,0.059035,14.924768,NaN,NaN,0.033284,U1,Not Walking


In [20]:
# split user = ['Luca' 'Pierpaolo' 'IvanHeibi']
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U3')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U3')&(dataset['user'] != 'U4')]
print(len(dataset))
print(len(dataset_test))
print(len(dataset_train))

5893
829
5064


In [21]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
3369,9.507575,10.234165,0.158989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U6,Not Walking
3759,9.121597,9.480357,0.104459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.361127,57.361127,57.361127,NaN,NaN,NaN,NaN,NaN,U8,Not Walking
385,10.185213,10.212387,0.007442,0.006686,0.034933,0.002702,0.018750,0.995120,0.995235,NaN,102.104874,57.808420,57.808420,57.808420,0.006653,NaN,NaN,NaN,0.006630,U1,Not Walking
1588,8.884712,10.625688,0.419922,0.073287,0.053265,0.048452,0.787671,0.899967,0.901489,NaN,57.890070,89.227269,89.227269,89.227269,0.065955,32.650145,NaN,NaN,0.002802,U12,Not Walking
2843,8.942786,16.534463,2.577438,1.437646,0.953046,0.996663,4.708989,0.984064,0.995174,NaN,151.325851,89.743050,89.743050,89.743050,1.414736,7.063483,NaN,NaN,1.195762,U1,Walking
5492,9.668824,9.961867,0.066162,0.007353,0.046467,0.002638,0.115185,0.208975,0.208975,NaN,180.903731,71.157359,71.157359,71.157359,0.001537,NaN,NaN,NaN,0.000409,U7,Not Walking
1404,8.567716,11.760600,0.624157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.937690,65.937690,65.937690,NaN,NaN,NaN,NaN,NaN,U6,Not Walking
841,8.769273,10.716209,0.391719,0.030694,0.038087,0.022782,0.052810,0.851706,0.865813,NaN,56.752269,87.353081,87.353081,87.353081,0.026142,1.851907,NaN,NaN,0.001463,U12,Not Walking
203,9.165269,11.523676,0.552401,0.028617,0.035273,0.007383,0.901806,0.351942,0.359661,NaN,166.025650,NaN,NaN,NaN,0.010071,27.279623,NaN,NaN,0.006792,U1,Not Walking
1003,5.701841,15.560615,3.279261,1.049208,1.036178,0.685455,5.040096,0.762588,0.803558,NaN,92.338516,87.499364,87.499364,87.499364,0.800113,5.040096,NaN,NaN,0.643599,U1,Walking


In [22]:
#dataset_train = dataset_train.fillna(0)

In [23]:
dataset_test.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
33,9.764513,14.028126,1.283204,0.615399,0.778458,0.655962,1.721314,0.955202,0.955202,NaN,36.378866,77.064227,77.064227,77.064227,0.587830,2.050172,NaN,NaN,0.554815,U3,Walking
49,3.324387,14.608536,4.031835,0.597423,0.619693,0.167046,4.223926,0.934642,0.937574,NaN,87.072128,77.221125,77.221125,77.221125,0.558377,NaN,NaN,NaN,0.555901,U2,Walking
51,4.595131,17.636696,3.551271,NaN,NaN,NaN,2.791507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U4,Walking
54,8.712177,9.963264,0.361961,0.447802,0.334574,0.116564,0.958200,0.983561,0.989576,NaN,77.496043,74.178611,74.178611,74.178611,0.440440,0.000000,NaN,NaN,0.393798,U3,Not Walking
56,5.371977,15.088214,3.438894,0.550989,0.749501,0.250989,3.610910,0.802652,0.823187,NaN,72.877824,78.588379,80.072657,79.330518,0.442252,4.333092,NaN,NaN,0.441829,U2,Walking


In [24]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [25]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            
            'linear_acceleration_mean',
            
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min',
            'sound_max',
            'sound_mean',
            
            
            'rv_gyro_mean',
            'lin_speed_mean',
            'rv_gyro_prox_mean',
            'lin_speed_prox_mean',
            'rv_gyro__grv_mean',
            ]


In [26]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.KNNImputer(n_neighbors=3)),
  ('Normalizer',RobustScaler())
  
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=3)),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'linear_acceleration_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min', 'sound_max', 'sound_mean',
                                  'rv_gyro_mean', 'lin_speed_mean',
                                  'rv_gyro_prox_mean', 'lin_speed_prox_mean',
                                  'rv_gyro__grv_mean'])])

In [27]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [28]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('Normalizer',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'linear_acceleration_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min', 'sound_max',
                                                   'sound_mean', 'rv_gyro_mean',
                                                   'lin_speed_mean',
                                                   'rv_gyro_prox_mean',
                                                   'lin_speed_prox_mean',
                                                   'rv_gyro__grv_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [29]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_test = dataset_test.drop(['target'], axis=1)
y_test = dataset_test.target

In [30]:
x_train.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean'],
      dtype='object')

In [31]:
from sklearn import model_selection
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_train, y_train,
    test_size=0.2,
    stratify = y_train,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:41:48] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.018722
0:	learn: 0.6542189	total: 75.2ms	remaining: 1m 15s
1:	learn: 0.6143324	total: 85.4ms	remaining: 42.6s
2:	learn: 0.5766971	total: 103ms	remaining: 34.2s
3:	learn: 0.5442490	total: 116ms	remaining: 28.9s
4:	learn: 0.5063864	total: 152ms	remaining: 30.3s
5:	learn: 0.4745664	total: 164ms	remaining: 27.2s
6:	learn: 0.4494496	total: 179ms	remaining: 25.4s
7:	learn: 0.4247556	total: 209ms	remaining: 25.9s
8:	learn: 0.4046774	total: 222ms	remaining: 24.5s
9:	learn: 0.3847194	total: 239ms	remaining: 23.6s
10:	learn: 0.3575781	total: 249ms	remaining: 22.4s
11:	learn: 0.3400173	total: 263ms	remaining: 21.7s
12:	learn: 0.3212213	total: 272ms	remaining: 20.7s
13:	learn: 0.3054747	total: 289ms	remaining: 20.3s
14:	learn: 

159:	learn: 0.0210414	total: 2.45s	remaining: 12.9s
160:	learn: 0.0209614	total: 2.46s	remaining: 12.8s
161:	learn: 0.0208628	total: 2.48s	remaining: 12.8s
162:	learn: 0.0207672	total: 2.49s	remaining: 12.8s
163:	learn: 0.0206800	total: 2.51s	remaining: 12.8s
164:	learn: 0.0205922	total: 2.52s	remaining: 12.7s
165:	learn: 0.0204888	total: 2.53s	remaining: 12.7s
166:	learn: 0.0204051	total: 2.54s	remaining: 12.7s
167:	learn: 0.0202607	total: 2.55s	remaining: 12.6s
168:	learn: 0.0201191	total: 2.57s	remaining: 12.6s
169:	learn: 0.0200059	total: 2.58s	remaining: 12.6s
170:	learn: 0.0199494	total: 2.58s	remaining: 12.5s
171:	learn: 0.0198811	total: 2.6s	remaining: 12.5s
172:	learn: 0.0197764	total: 2.61s	remaining: 12.5s
173:	learn: 0.0197165	total: 2.62s	remaining: 12.5s
174:	learn: 0.0195941	total: 2.63s	remaining: 12.4s
175:	learn: 0.0195214	total: 2.65s	remaining: 12.4s
176:	learn: 0.0194527	total: 2.67s	remaining: 12.4s
177:	learn: 0.0193769	total: 2.68s	remaining: 12.4s
178:	learn: 0

328:	learn: 0.0112195	total: 4.91s	remaining: 10s
329:	learn: 0.0111977	total: 4.93s	remaining: 10s
330:	learn: 0.0111848	total: 4.94s	remaining: 9.99s
331:	learn: 0.0111338	total: 4.96s	remaining: 9.97s
332:	learn: 0.0110889	total: 4.97s	remaining: 9.95s
333:	learn: 0.0110623	total: 4.98s	remaining: 9.94s
334:	learn: 0.0110414	total: 5s	remaining: 9.92s
335:	learn: 0.0109984	total: 5s	remaining: 9.89s
336:	learn: 0.0109939	total: 5.02s	remaining: 9.87s
337:	learn: 0.0109464	total: 5.03s	remaining: 9.85s
338:	learn: 0.0109187	total: 5.04s	remaining: 9.82s
339:	learn: 0.0108866	total: 5.05s	remaining: 9.8s
340:	learn: 0.0108618	total: 5.06s	remaining: 9.77s
341:	learn: 0.0108393	total: 5.07s	remaining: 9.75s
342:	learn: 0.0108013	total: 5.09s	remaining: 9.74s
343:	learn: 0.0107682	total: 5.11s	remaining: 9.75s
344:	learn: 0.0107315	total: 5.12s	remaining: 9.73s
345:	learn: 0.0107127	total: 5.14s	remaining: 9.72s
346:	learn: 0.0106783	total: 5.15s	remaining: 9.7s
347:	learn: 0.0106499	to

498:	learn: 0.0068726	total: 7.36s	remaining: 7.38s
499:	learn: 0.0068691	total: 7.37s	remaining: 7.37s
500:	learn: 0.0068535	total: 7.38s	remaining: 7.36s
501:	learn: 0.0068497	total: 7.42s	remaining: 7.36s
502:	learn: 0.0068289	total: 7.44s	remaining: 7.35s
503:	learn: 0.0068197	total: 7.46s	remaining: 7.34s
504:	learn: 0.0068040	total: 7.48s	remaining: 7.33s
505:	learn: 0.0067936	total: 7.51s	remaining: 7.33s
506:	learn: 0.0067837	total: 7.53s	remaining: 7.32s
507:	learn: 0.0067673	total: 7.56s	remaining: 7.32s
508:	learn: 0.0067495	total: 7.57s	remaining: 7.3s
509:	learn: 0.0067230	total: 7.59s	remaining: 7.29s
510:	learn: 0.0067135	total: 7.6s	remaining: 7.27s
511:	learn: 0.0067035	total: 7.63s	remaining: 7.28s
512:	learn: 0.0066892	total: 7.66s	remaining: 7.27s
513:	learn: 0.0066598	total: 7.68s	remaining: 7.26s
514:	learn: 0.0066434	total: 7.7s	remaining: 7.25s
515:	learn: 0.0066359	total: 7.73s	remaining: 7.25s
516:	learn: 0.0066218	total: 7.75s	remaining: 7.24s
517:	learn: 0.0

665:	learn: 0.0048764	total: 10.2s	remaining: 5.13s
666:	learn: 0.0048647	total: 10.3s	remaining: 5.12s
667:	learn: 0.0048566	total: 10.3s	remaining: 5.1s
668:	learn: 0.0048405	total: 10.3s	remaining: 5.09s
669:	learn: 0.0048365	total: 10.3s	remaining: 5.08s
670:	learn: 0.0048237	total: 10.3s	remaining: 5.06s
671:	learn: 0.0048111	total: 10.3s	remaining: 5.04s
672:	learn: 0.0047910	total: 10.3s	remaining: 5.03s
673:	learn: 0.0047875	total: 10.4s	remaining: 5.01s
674:	learn: 0.0047838	total: 10.4s	remaining: 4.99s
675:	learn: 0.0047758	total: 10.4s	remaining: 4.98s
676:	learn: 0.0047698	total: 10.4s	remaining: 4.96s
677:	learn: 0.0047594	total: 10.4s	remaining: 4.95s
678:	learn: 0.0047522	total: 10.4s	remaining: 4.93s
679:	learn: 0.0047454	total: 10.4s	remaining: 4.92s
680:	learn: 0.0047380	total: 10.5s	remaining: 4.9s
681:	learn: 0.0047311	total: 10.5s	remaining: 4.88s
682:	learn: 0.0047221	total: 10.5s	remaining: 4.87s
683:	learn: 0.0047160	total: 10.5s	remaining: 4.85s
684:	learn: 0.

837:	learn: 0.0036305	total: 12.9s	remaining: 2.5s
838:	learn: 0.0036233	total: 12.9s	remaining: 2.48s
839:	learn: 0.0036110	total: 12.9s	remaining: 2.47s
840:	learn: 0.0036008	total: 13s	remaining: 2.45s
841:	learn: 0.0035988	total: 13s	remaining: 2.44s
842:	learn: 0.0035974	total: 13s	remaining: 2.42s
843:	learn: 0.0035961	total: 13s	remaining: 2.4s
844:	learn: 0.0035883	total: 13s	remaining: 2.39s
845:	learn: 0.0035870	total: 13s	remaining: 2.37s
846:	learn: 0.0035857	total: 13.1s	remaining: 2.36s
847:	learn: 0.0035790	total: 13.1s	remaining: 2.34s
848:	learn: 0.0035714	total: 13.1s	remaining: 2.33s
849:	learn: 0.0035637	total: 13.1s	remaining: 2.31s
850:	learn: 0.0035607	total: 13.1s	remaining: 2.29s
851:	learn: 0.0035542	total: 13.1s	remaining: 2.28s
852:	learn: 0.0035529	total: 13.2s	remaining: 2.27s
853:	learn: 0.0035516	total: 13.2s	remaining: 2.25s
854:	learn: 0.0035453	total: 13.2s	remaining: 2.24s
855:	learn: 0.0035387	total: 13.2s	remaining: 2.22s
856:	learn: 0.0035343	tota

,Model,Accuracy,Bal Acc.,Time
1,AdaBoost,75.995175,79.103127,6.479115
2,Skl GBM,75.874548,79.599134,8.790603
3,Skl HistGBM,75.753920,79.441901,8.594050
4,LightGBM,74.909530,78.816355,6.150263
5,CatBoost,74.185766,78.288656,21.349098
6,XGBoost,73.944511,77.855420,6.683483
7,Extra Trees,71.411339,76.216323,5.864582
8,Random Forest,70.808203,75.786471,6.794546
9,Decision Tree,69.481303,74.116297,6.545350


In [32]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[305  13]
 [201 310]]
              precision    recall  f1-score   support

 Not Walking       0.60      0.96      0.74       318
     Walking       0.96      0.61      0.74       511

    accuracy                           0.74       829
   macro avg       0.78      0.78      0.74       829
weighted avg       0.82      0.74      0.74       829



In [34]:
model = tree_classifiers["AdaBoost"]
model.fit(x_train, y_train)
pred = model.predict(x_test) 

In [38]:
# save the model to disk
import pickle
filename = 'model_walking_not_walking_v1.sav'
pickle.dump(model, open(filename, 'wb'))